In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_correct_incorrect_player* function**:
* Input: players_info dictionary
* Displays bar chart showing for each user the number of correct and incorrect alternatives selected in multiple-choice questions

In [1]:
outputAltPerPlayer=widgets.Output()
def vis_correct_incorrect_player():
    out=interactive(construct_correct_incorrect_player,
             #constructing a dropdown to select how many data we want to see in visualisations for x axis
            xaxis=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                                description='x axe',
                                disabled=False),
             #constructing a dropdown to order for alphabetic keys or for values
            order=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                                description='Order',
                                disabled=False),
             percentagePerPlayer=False,
             orderLabelValues=[("Order by all true and false responses values",0),
                               ("Order by true responses values","true"),
                               ("Order by false responses values","false")])
    md_text=widgets.HTML("<h4>Bar chart with number of correct and incorrect answers per player</h4>")
    with outputAltPerPlayer:
        print("Updating visualisation with the current selection of players and alternatives...")
    return widgets.VBox([md_text,out, outputAltPerPlayer])

def construct_correct_incorrect_player(xaxis,order, percentagePerPlayer, orderLabelValues):
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
    true_false_per_player = {}
    total_max=0
    empty=True
    for player in playersSelected:
        true_false_per_player[player]={}
        if len(players_info[player]["alternatives"].keys())>0:
            total_true = 0
            total_false = 0
            for alt in alternatives_list_selected:
                _, results = zip(*players_info[player]["alternatives"][alt]) # take only T/F result values
                total_true += results.count("True") + results.count(True)
                total_false += results.count("False") +results.count(False)
                if total_true+total_false>total_max:
                    total_max=total_true+total_false
            true_false_per_player[player]["true"]=total_true
            true_false_per_player[player]["false"]=total_false
            empty=False
        else:
            true_false_per_player[player]["true"]=0
            true_false_per_player[player]["false"]=0
    total_max+=0.5
    if orderLabelValues==0:
        lambdafunc=lambda item:item[1]["true"]+item[1]["false"]
    else:
        lambdafunc=lambda item:item[1][orderLabelValues]
    true_false_per_player = orderDict(true_false_per_player,order,lambdafunc)
    nbgraphshown, true_false_per_player,N = organizeDictPerxAxisSelected(playersSelected,xaxis, true_false_per_player)
    for n in range(nbgraphshown):
        if not empty:
            true_per_player=[true_false_per_player[n][player]["true"] for player in true_false_per_player[n].keys()]
            false_per_player=[true_false_per_player[n][player]["false"] for player in true_false_per_player[n].keys()]
            list_player=[player for player in true_false_per_player[n].keys()]
            if n<nbgraphshown-1:
                ind = np.arange(N)
            else:
                ind = np.arange(len(list_player))
        if percentagePerPlayer:
            # Bar chart showing for each user the percentage of incorrect alternatives
            # selected in multiple-choice questions
            if not empty:
                get_percentage_bar(true_per_player,false_per_player, list_player)
            else:
                noDataToFillVis(10)
            ylabel="percentage of good response"
            title="Percentage of good responses per player"
        else:
            # Bar chart showing for each user the number of correct and incorrect alternatives
            # selected in multiple-choice questions
            if not empty:
                get_two_superposed_values_bars(ind,list_player,
                                           true_per_player, "Correct",'g',
                                           false_per_player, "Incorrect",'r')
            else:
                noDataToFillVis(10)
            ylabel="Number of response"
            title="Correct and incorrect responses per player"
        plt.ylabel(ylabel)
        plt.xlabel("Player")
        plt.xticks(rotation=90)
        plt.title(title)
        plt.show()
    with outputAltPerPlayer:
        clear_output(wait=False)